In [4]:
import json
with open('/mnt/d/zaloAI2020/zac2020/data/release/private_test/private_test/private_test.jsonl',encoding="utf-8") as f:
    p_test = [json.loads(jline) for jline in f.read().splitlines()]
len(p_test)

278

## Team names - NER

In [5]:
from vncorenlp import VnCoreNLP

# To perform word segmentation, POS tagging, NER
annotator = VnCoreNLP("/home/hoanganh/zac2020-IE/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g') 

In [6]:
text = "Cú đúp bàn thắng của tiền đạo Suso cùng các pha lập công của Franck Kessie và Samu Castillejo đã giúp AC Milan có trận đại thắng 4-1 trước chủ nhà Sassuolo, qua đó tạm thời giúp chiếc ghế HLV của Gennaro Gattuso tạm yên ổn."
annotate = annotator.annotate(text)

In [47]:
parse

[[('sub', 4, 1),
  ('nmod', 1, 2),
  ('adv', 4, 3),
  ('root', 0, 4),
  ('loc', 4, 5),
  ('pob', 5, 6),
  ('nmod', 6, 7),
  ('nmod', 6, 8),
  ('punct', 4, 9)],
 [('sub', 9, 1),
  ('nmod', 1, 2),
  ('punct', 1, 3),
  ('nmod', 1, 4),
  ('nmod', 4, 5),
  ('nmod', 5, 6),
  ('punct', 1, 7),
  ('adv', 9, 8),
  ('root', 0, 9),
  ('loc', 9, 10),
  ('pob', 10, 11),
  ('punct', 9, 12)]]

In [8]:
def filter_ner_annotation(ner):
    output=[]
    type=text=''
    for sentence in ner:
        for token in sentence:
            if token[1][0]=='B':
                if text!='':
                    output.append((type,text.strip()))
                type=token[1][2:]
                text = ' '.join(token[0].split('_'))+' '
            elif token[1]=='O':
                if text!='':
                    output.append((type,text.strip()))
                type = text = ''
            else:
                text += ' '.join(token[0].split('_'))+' '
    if text!='':
        output.append((type,text))

    return output

def count_ner(ner_output):
    ner_dict={}
    for token in ner_output:
        if token not in ner_dict:
            ner_dict[token]=1
        else:
            ner_dict[token]+=1
    return ner_dict

def two_most_frequent_names(ner_dict,tag={'PER','ORG'}):
    org = [(name,ner_dict[name]) for name in ner_dict if name[0]=='ORG']
    sorted_org = [item[0][1] for item in sorted(org, key=lambda item: -item[1])]

    if len(sorted_org)>=2:
        return sorted_org[:2]
    else:
        per = [(name,ner_dict[name]) for name in ner_dict if name[0]=='PER']
    sorted_per = [item[0][1] for item in sorted(per, key=lambda item: -item[1])]
    return sorted_org+sorted_per[:2-len(sorted_org)]

In [9]:
text = ('Do đụng phải đối thủ được đánh giá cao hơn, không ngạc Young Boys chọn lối chơi phòng ngự chặt chẽ và chờ cơ hội phản công mỗi khi có cơ hội. Chủ trương của họ phát huy hiệu quả khá tốt trong những phút đầu.'
'Việc các cầu thủ ở tuyến giữa M.U như Fred hay Pogba không có nhiều thời gian xử lý khi có tới vài đối thủ ập vào mỗi khi có bóng khiến Quỷ đỏ gặp không ít khó khăn. Thực tế là thủ môn David De Gea của họ đã có những giây phút phải làm việc vất vả khi đối mặt với những cú đá tầm xa hay các pha tạt bóng ẩn chứa đầy hiểm họa.'
'Rất may cho M.U là trong tình cảnh như vậy, Pogba đã giúp các vị khách vượt qua sóng gió khi tỏa sáng đúng lúc ở phút 35. Với cú "nã đại bác" từ ngoài vòng cấm làm bó tay thủ thành David von Ballmoos, tiền vệ người Pháp đã chứng minh tại sao anh là một trong những cầu thủ đắt giá nhất thế giới.'
'9 phút sau, M.U có cơ hội rất tốt để nhân đôi cách biệt khi hậu vệ Kevin Mbabu của Young Boys để bóng chạm tay trong vòng cấm sau đường chuyền của cầu thủ mới trở lại sau chấn thương Luke Shaw. Trên chấm 11m, Pogba dễ dàng hoàn tất cú đúp giúp M.U tạo ra cách biệt an toàn.'
'Tạo ra lợi thế không nhỏ ngay trong hiệp 1, M.U thi đấu khá ung dung trong hiệp 2. Họ không cần phải đẩy tốc độ trận đấu lên quá cao trong khi Young Boys buộc phải nỗ lực tìm kiếm bàn gỡ. Phút 66, lại là Pogba ghi dấu ấn với đường chuyền giúp tiền đạo Martial thoát xuống dứt điểm vào ngay góc gần mang về bàn thắng thứ 3 cho M.U.'

'Trong những phút còn lại, Young Boys tỏ ra buông xuôi và tất cả những gì họ làm được chỉ là vài cú dứt điểm không đủ khó để khiến khung thành đội khách phải chao đảo. Về phần mình, M.U cũng chỉ thi đấu cầm chừng và tỷ số 3-0 được giữ nguyên cho tới khi trọng tài thổi còi mãn cuộc.')

ner=annotator.ner(text)

In [10]:
two_most_frequent_names(count_ner(filter_ner_annotation(ner)))

['M.U', 'Young Boys']

In [11]:
def item_to_string(item):
    s=''
    for para in item['original_doc']['_source']['body']:
        s+=para['text']
    return s

## Scores - regular expression

In [12]:
import re
def detect_score_board(text):
    candidates = re.findall('[^-][0-9]+\s*-\s*[0-9]+[^-]',text)
    if len(candidates)==0:
        return (0,0)
    scores_numbers = [int(number_str) for number_str in re.findall('[0-9]+',candidates[-1])]
    scores_numbers.sort()
    return scores_numbers[1], scores_numbers[0]


In [13]:
detect_score_board(text)

(3, 0)

## Output

In [15]:
import codecs
with codecs.open('submit_private.jsonl', 'w', "utf-8") as modified:
    for item in p_test:
        text = item_to_string(item)
        ner = annotator.ner(text)
        team1,team2=two_most_frequent_names(count_ner(filter_ner_annotation(ner)))
        score1,score2 = detect_score_board(text)
        json.dump({"test_id": item['test_id'], "match_summary": {"players": {"team1": team1, "team2": team2}, "score_board": {"score1": str(score1), "score2": str(score2)}, "score_list": [{"player_name": "", "time": "", "team": ""}], "card_list": [{"player_name": "", "time": "", "team": ""}], "substitution_list": [{"player_in": "", "time": "", "player_out": ""}]}}, modified,ensure_ascii=False)
        modified.write('\n')